# Persamaan Parabolic 2D menggunakan Metode *Forward Euler* (Explicit Time Marching)

Diketahui persamaan yang akan diselesaikan sebagai berikut ini

\begin{equation}
\dfrac{\partial u}{\partial t} = \dfrac{\partial^2 u}{\partial x^2} + \dfrac{\partial^2 u}{\partial y^2} - f(x,y,t)
\end{equation}

akan diterapkan pada domain bujur sangkar $0\le x,y \le 1$, dengan suku sumber dinyatakan dengan

\begin{align} 
f(x,y,t) = &1000 \left[2\sinh\left(x-\frac{1}{2}\right) + 4\left(x-\frac{1}{2}\right)\cosh\left(x-\frac{1}{2}\right) + \left(x-\frac{1}{2}\right)^2\sinh\left(x-\frac{1}{2}\right)\right] + \nonumber\\
& 1000 \left[2\sinh\left(y-\frac{1}{2}\right) + 4\left(y-\frac{1}{2}\right)\cosh\left(y-\frac{1}{2}\right) + \left(y-\frac{1}{2}\right)^2\sinh\left(y-\frac{1}{2}\right)\right] 
\end{align}

Syarat batas ditetapkan sebagai berikut:

\begin{align} 
u(t,0,y) & = 1000\left[ \frac{1}{4}\sinh \left( -\frac{1}{2}\right) + \left( y-\frac{1}{2}\right)^2\sinh\left(y-\frac{1}{2}\right) \right] \\
u(t,1,y) & = 1000\left[ \frac{1}{4}\sinh \left( \frac{1}{2}\right) + \left( y-\frac{1}{2}\right)^2\sinh\left(y-\frac{1}{2}\right) \right] \\
u(t,x,0) & = 1000\left[  \left( x-\frac{1}{2}\right)^2\sinh\left(x-\frac{1}{2}\right) + \frac{1}{4}\sinh \left( -\frac{1}{2}\right) \right] \\
u(t,x,1) & = 1000\left[  \left( x-\frac{1}{2}\right)^2\sinh\left(x-\frac{1}{2}\right) + \frac{1}{4}\sinh \left( \frac{1}{2}\right) \right] 
\end{align}

Syarat awal yang digunakan adalah $$u(0,x,y) = 0$$.

Tentukan solusi menggunakan metode beda hingga.  Solusi analitik  untuk kondisi *steady* adalah berikut.

\begin{equation}
u(\infty,x,y)  = 1000\left[ \left( x-\frac{1}{2}\right)^2\sinh\left(x-\frac{1}{2}\right) + \left( y-\frac{1}{2}\right)^2\sinh\left(y-\frac{1}{2}\right) \right]
\end{equation}

# Diskretisasi

Kita lakukan diskretisasi ruang menggunakan metode beda hingga (*finite difference*).

$$\left.\frac{\partial^2 u}{\partial x^2}\right|_{i,j,n} \approx \frac{u_{i+1,j,n} - 2u_{i,j,n}+u_{i-1,j,n}}{\Delta x^2}$$

$$\left.\frac{\partial^2 u}{\partial y^2}\right|_{i,j,n} \approx \frac{u_{i,j+1,n} - 2u_{i,j,n}+u_{i,j-1,n}}{\Delta y^2}$$

Indeks $i$ dan $j$ menyatakan ruang dan indeks $n$ menyatakan waktu.

Diskretisasi waktu dilakukan dengan menggunakan turunan pertama terhadap waktu:

$$ \left. \frac{\partial u}{\partial t} \right|_{i,j,n} = \dfrac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t} $$


Dengan mensubstitusi persamaan diferensi di atas ke persamaan diferensial, akan didapatkan

$$\dfrac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t}  = \left(\frac{u_{i+1,j,n} - 2u_{i,j,n}+u_{i-1,j,n}}{\Delta x^2}\right) + \left(\frac{u_{i,j+1,n} - 2u_{i,j,n}+u_{i,j-1,n}}{\Delta y^2}\right) - f_{i,j,n}$$

atau

$$\dfrac{u_{i,j,n+1} - u_{i,j,n}}{\Delta t}  =  \frac{1}{\Delta x^2} u_{i-1,j,n} + \frac{1}{\Delta x^2} u_{i+1,j,n} - \left(\dfrac{2}{\Delta x^2} + \dfrac{2}{\Delta y^2} \right) u_{i,j,n}  + \frac{1}{\Delta y^2} u_{i,j-1,n} + \frac{1}{\Delta y^2} u_{i,j+1,n} - f_{i,j,n}$$

Persamaan diferensi kemudian dapat kita susun sebagai berikut

\begin{align}
 u_{i,j,n+1}  = & \Delta t \left[ \left(  a_W u_{i-1,j,n} + a_E u_{i+1,j,n} + a_O u_{i,j,n}  + a_S u_{i,j-1,n} + a_N u_{i,j+1,n} \right) \right] \\ & - \Delta t f_{i,j,n} 
\end{align}

di mana  $a_0, a_W, a_E, a_S$ dan $a_N$ adalah koefisien-koefisien untuk titik-titik O, West (kiri), East (kanan), South (bawah) dan North (atas). Lihat definisi posisi pada slide kuliah.

Dari persamaan di atas dapat dilihat bahwa nilai-nilai koefisien titik adalah sebagai berikut:

\begin{align}
a_0 &= -\left(\frac{2}{\Delta x^2} + \frac{2}{\Delta y^2} - \dfrac{1}{\Delta t} \right) \\
a_W &= a_E = \frac{1}{\Delta x^2} \\
a_S &= a_N = \frac{1}{\Delta y^2}
\end{align}




# Coding

Pertama kita tentukan terlebih dahulu banyaknya interval dan menginisialisasi semua variabel yang diperlukan.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import math
import time
from PIL import Image
import imageio
import os

In [ ]:
## LANGKAH 1 :  Parameter input dan inisialisasi

N = 10                             # banyaknya interval arah x
M = 10                            # banyaknya interval arah y
T = 10                            # banyaknya interval waktu

xmin = 0                           # batas bawah domain arah x
xmax = 1                           # batas atas domain arah x

ymin = 0                           # batas bawah domain arah y
ymax = 1                           # batas atas domain arah y

tmin = 0
tmax = 1

# tentukan lebar interval
dx = ...              
dy = ...
dt = ...

# nilai kuadrat dari interval
dx2 = dx**2
dy2 = dy**2

# nilai diskret dari x dan y
x = np.linspace(..., ..., ...)
y = np.linspace(..., ..., ...)
t = np.linspace(..., ..., ...)

# inisialisasi

f = np.zeros((..., ..., ...))                 # matriks f
u = np.zeros((..., ..., ...))                # matriks u 



uanal = np.zeros((..., ...))             # matriks solusi analitis untuk plotting 2 dimensi
uplot = np.zeros((..., ...))             # matriks solusi komputasi untuk plotting 2 dimensi


In [ ]:
## LANGKAH 2 : Penentuan syarat awal dan syarat batas

# IC

u[0,:,:] = 0

# BC

for j in range(...):
    u[:,0,j] = ... # BC u(0,y) / kiri domain
    u[:,N,j] = ...  # BC u(1,y) / kanan domain
    
for i in range(...):
    u[:,i,0] = ... # BC u(x,0) / bawah domain
    u[:,i,M] = ...   # BC u(x,1) / atas domain


In [ ]:
## LANGKAH 3 : Penentuan suku sumber

# source
for i in range(..., ...):
    for j in range(..., ...):
        f[:,i,j] = ...

In [ ]:
## LANGKAH 4 :  Penentuan koefisien a

aO = ...
aW = ...
aE = ...
aN = ...
aS = ...

In [ ]:
## LANGKAH 5 :

# Cari solusi secara eksplisit

tic = time.time()
for n in range(...):
    for j in range(..., ...):
        for i in range(..., ...):
            u[n+1,i,j] = ...
toc = time.time()
print('Waktu = ',toc-tic)


In [ ]:
## LANGKAH 6 :

# cari solusi analitik

for i in range(...):
    for j in range(...):
        uanal[i,j] = ...



In [ ]:
## LANGKAH 7 :  buat plot

# tentukan time step untuk diplot
n = ...

# Persiapan plotting
X, Y = np.meshgrid(x,y)             # susun meshgrid untuk plot 3D
fig = plt.figure(figsize=(15,25))
cmap = mpl.cm.get_cmap('seismic')   # tentukan colormap sesuai keinginan



# Transpose untuk menyesuaikan plot
uplot = u[...,...,...].T    
uanal = uanal.T
galat = uplot - uanal


# subplot pertama menampilkan hasil dalam bentuk plot 2D
ax = fig.add_subplot(3,2,1)
c = ax.pcolor(X,Y, uplot, cmap=cmap,shading='auto')
ax.set_xlabel(r"x", fontsize=12)
ax.set_ylabel(r"y", fontsize=12)
ax.set_title(f'Hasil Komputasi, t = {t[n]:.3f} detik')

# subplot kedua menampilkan hasil dalam bentuk plot 3D
ax = fig.add_subplot(3,2,2, projection='3d')
p = ax.plot_surface(X,Y, uplot, linewidth=0, cmap=cmap)
ax.set_xlabel(r"x", fontsize=12)
ax.set_ylabel(r"y", fontsize=12)
ax.set_title(f'Hasil Komputasi, t = {t[n]:.3f} detik')
cb = plt.colorbar(p, ax=ax, shrink=0.75)
cb.set_label(r"u(x,y)", fontsize=12)

# subplot ketiga menampilkan solusi analitik dalam bentuk plot 2D
ax = fig.add_subplot(3,2,3) 
c = ax.pcolor(X,Y,uanal, cmap=cmap,shading='auto')
ax.set_xlabel(r"x", fontsize=12)
ax.set_ylabel(r"y", fontsize=12)
ax.set_title("Solusi Analitik Steady State")

# subplot keempat menampilkan solusi analitik dalam bentuk plot 3D
ax = fig.add_subplot(3,2,4, projection='3d')
p = ax.plot_surface(X,Y, uanal, linewidth=0, cmap=cmap)
ax.set_xlabel(r"x", fontsize=12)
ax.set_ylabel(r"y", fontsize=12)
ax.set_title("Solusi Analitik Steady State")
cb = plt.colorbar(p, ax=ax, shrink=0.75)
cb.set_label(r"u(x,y)", fontsize=12)


# subplot kelima menampilkan kontur dari galat
ax = fig.add_subplot(3,2,5)
contours = plt.contour(X,Y,galat, 20, colors='black')
plt.clabel(contours, inline=True, fontsize=8)
ax.set_xlabel("x", fontsize=12)
ax.set_ylabel("y", fontsize=12)
ax.set_title("Kontur dari Galat")

# subplot keenam menampilkan galat dalam bentuk plot 3D
ax = fig.add_subplot(3,2,6, projection='3d')
p = ax.plot_surface(X,Y, galat, linewidth=0, cmap=cmap)
ax.set_xlabel(r"x", fontsize=12)
ax.set_ylabel(r"y", fontsize=12)
ax.set_title("Galat")
cb = plt.colorbar(p, ax=ax, shrink=0.75)
cb.set_label(r"galat(x,y)", fontsize=12)

In [ ]:
## LANGKAH 8 :  Buat file PNG untuk animasi

cmap = mpl.cm.get_cmap('seismic')
n = T
#n = 20

files = []

for i in range(n):

    fig = plt.figure(figsize=(8,8))
    ax = plt.axes(projection ='3d')
    X, Y = np.meshgrid(x,y)  
    
    uplot = u[i,:,:].T

    p = ax.plot_surface(X,Y, uplot, linewidth=0, cmap=cmap)
    ax.set_xlabel(r"x", fontsize=12)
    ax.set_ylabel(r"y", fontsize=12)
    ax.set_zlabel(r"u (x,y)", fontsize=12)
    ax.set_title(f'Hasil Komputasi, t = {t[i]:.3f} detik')

    
    filename = '3d_vis_eksplisit_' + str(i) + '.png'   # file PNG di masing-masing timestep
    plt.savefig(filename,dpi=75)
    files.append(filename)
    plt.close()

In [ ]:
## LANGKAH 9 : buat file GIF untuk animasi

frames = []

for i in range(T):
    image = imageio.v2.imread(f'3d_vis_eksplisit_{i}.png')
    frames.append(image)

imageio.mimsave('3d_vis_eksplisit.gif', # output gif
                frames,                 # array of input frames
                fps = 30)               # optional: frames per second

In [ ]:
## LANGKAH 10 : Hapus file PNG yang tidak diperlukan


for i in range(T):
    os.remove(f'3d_vis_eksplisit_{i}.png')